# 1. Import & Setting

### 1-1) Path Setting

In [1]:
import os
project_path = "C:/workspace/Bus Project"
os.chdir(project_path)

### 1-2) Import

In [2]:
import datetime
import numpy as np
import pandas as pd
import multiprocessing
from functools import partial
from multiprocessing import Pool

import bus.analyzer as anz

### 1-3) 한글 폰트 설정

In [3]:
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

# 2. 이용자 & 정류장 추출

### 2-1) - 이용데이터 로딩

In [ ]:
# 데이터 기간 설정
start_date = datetime.datetime(2019, 6, 1)
end_date = datetime.datetime(2019, 8, 29)

# 로딩할 파일 명 리스트 생성
input_path_list = anz.make_input_path(start_date, end_date)

# 전체 데이터 로딩 (multi)
usage_df = anz.parallel_load_total_usage_df(input_path_list)

### 2-2) - 정류장 추출 & 분석

In [5]:
#정류장 추출
station_df = anz.create_station_df(usage_df)
print("추출된 정류장 개수: ",len(station_df))

# # 주소 컬럼 추가
station_df = anz.set_address_column(station_df)

# # sorting
station_df = station_df.sort_values(by="station_id")

# # storing
station_df.to_csv("data/analysis/station_df.csv", encoding="CP949", index=False)

  0%|                                                                                 | 1/3654 [00:00<06:29,  9.38it/s]

추출된 정류장 개수:  3654


100%|██████████████████████████████████████████████████████████████████████████████| 3654/3654 [04:25<00:00, 13.74it/s]


In [ ]:
station_df = anz.load_station_df()

### 2-3) - 이용자 추출 & 분석

In [ ]:
# 유저 목록 생성
user_df = anz.create_user_df(usage_df)

# 이용 날짜 분석 완료
user_df = anz.analyze_usage_date(user_df, usage_df)

# 이용 횟수 분석 완료
user_df = anz.analyze_usage_num(user_df, usage_df)

# 이용 비율 분석 완료
user_df = anz.parallel_analyze_usage_ratio(user_df)

# 출발, 도작지 분석 완료
user_df = anz.analyze_start_end(user_df, usage_df)

### 2-4) 관광객 추출

In [ ]:
# case1, 2의 사용 정류장 추출 및 저장
both_station_df = anz.extract_used_station(user_df, usage_df, station_df, 'both')
both_station_df.to_csv("data/analysis/used_station_df.csv", encoding="cp949", index=False)

# HAND WORK - "data/case1_2.csv"를 보고 직접 관광 관련 정류장 컬럼을 삽입하고 "data/case1_2_filtered.csv"로 저장
tour_station_df = pd.read_csv("data/analysis/tour_station_df.csv", low_memory=False, encoding = "utf-8") #, dtype=dtype)

# 관광관련 정류장 방문 횟수 분석
user_df = anz.analyze_tour_station_visit(user_df, usage_df, tour_station_df)

# 이용자 케이스 분류
user_df = anz.parallel_analyze_case(user_df)

# 관광객 분석
user_df = anz.parallel_analyze_tourist(user_df)

### 2-5) 정류장 이용 분석 

In [ ]:
# 정류장 분석
station_df = anz.analyze_station_usage(station_df, usage_df, user_df)

### 2-6) 이용자 및 정류장 저장

In [ ]:
# 이용자 데이터 저장
user_df.to_csv("data/analysis/user_df.csv", encoding="cp949", index=False)

# 정류장 데이터 저장
station_df.to_csv("data/analysis/station_df.csv", encoding="CP949", index=False)

# 3. 관광객 추출 알고리즘 검증

### 3-1) 알고리즘 검증 - tour_visit검증

In [ ]:
plt = anz.draw_tour_visit_analysis_graph(user_df[user_df["period"]<15])
plt.show()

### 3-2) 알고리즘 검증 - case검증

In [ ]:
anz.draw_period_user_per_case(user_df)

### 3-3) 알고리즘 검증 - period검증

In [ ]:
plt = anz.draw_period_analysis_graph(user_df)
plt.show()

### 3-4) 알고리즘 검증 - usage_ratio검증

In [ ]:
columns = ['both', 'first', 'last', 'neither']
boundary = 90
for column in columns:
    down = len(user_df[(user_df[column] == True) & (user_df["usage_ratio"] < boundary)])
    up = len(user_df[(user_df[column] == True) & (user_df["usage_ratio"] >= boundary)])
    print("%7s 이용자수 = (%3d%%미만: %6d명, %3d%%이상: %6d명)" % (column, boundary, down, boundary, up))
plt = anz.draw_usage_ratio_analysis_graph(user_df)
plt.show()

# 4. 결과 출력 

### 3-1) 이용자 출력

In [ ]:
user_df

### 3-2) 정류장 출력

In [ ]:
station_df

### 3-3) 방문기간 별 관광객 수 출력

In [ ]:
df = user_df[user_df["tourist"] == True]
anz.draw_period_user(plt, df)

### 3-1) both 관광객 출력

In [ ]:
df = user_df[2<=user_df['period']][user_df["period"]<=15][user_df['both'] == True][user_df["tour_visit"]>3][user_df['usage'] > 5]
anz.show_od_pattern(df, usage_df, 100)

### 3-2) first 관광객 출력

In [ ]:
df = user_df[2<=user_df['period']][user_df["period"]<=15][user_df['first'] == True][user_df["tour_visit"]>3][user_df['usage'] > 5]
anz.show_od_pattern(df, usage_df, 100)

### 3-3) last 관광객 출력

In [ ]:
df = user_df[2<=user_df['period']][user_df["period"]<=15][user_df['last'] == True][user_df["tour_visit"]>3][user_df['usage'] > 5]
anz.show_od_pattern(df, usage_df, 100)

### 3-4) neither 관광객 출력

In [ ]:
df = user_df[2<=user_df['period']][user_df["period"]<=15][user_df['neither'] == True][user_df["tour_visit"]>3][user_df['usage'] > 5]
anz.show_od_pattern(df, usage_df, 100)